# Classification using Deep Learning with Histogram data

---

### Reading the data

First, we’ll load the saved image and label data from the NumPy files.

In [ ]:
import numpy as np

# Caminho base para os arquivos .npy
base_path = '/content/drive/MyDrive/cubsat'

# Carregamento dos arquivos com os nomes esperados
train_images = np.load(f'{base_path}/train_images.npy')
train_labels = np.load(f'{base_path}/train_labels.npy')
val_images   = np.load(f'{base_path}/val_images.npy')
val_labels   = np.load(f'{base_path}/val_labels.npy')

# Exibir os shapes para conferência
print("train_images shape:", train_images.shape)
print("train_labels shape:", train_labels.shape)
print("val_images shape:", val_images.shape)
print("val_labels shape:", val_labels.shape)


In [ ]:
import numpy as np

# Display image information
print("=== Image Characteristics ===")
print(f"Shape of training set: {train_images.shape} (Samples, Height, Width, Channels)")
print(f"Shape of validation set: {val_images.shape} (Samples, Height, Width, Channels)")

# Image resolution (height and width)
image_resolution = train_images.shape[1:3]  # Taking height and width dimensions
print(f"Image resolution: {image_resolution} pixels")

# Total number of images
print(f"Number of images in training set: {train_images.shape[0]}")
print(f"Number of images in validation set: {val_images.shape[0]}")

# Data type of the images
print(f"Data type of images: {train_images.dtype}")

# Range of values in the images
print(f"Minimum and maximum values in training dataset: {train_images.min()} to {train_images.max()}")

# Display some unique labels and their counts in the training set
unique_labels_train, counts_train = np.unique(train_labels, return_counts=True)
print("\n=== Class Distribution in Training Set ===")
for label, count in zip(unique_labels_train, counts_train):
    print(f"Class {label}: {count} samples")

# Display some unique labels and their counts in the validation set
unique_labels_val, counts_val = np.unique(val_labels, return_counts=True)
print("\n=== Class Distribution in Validation Set ===")
for label, count in zip(unique_labels_val, counts_val):
    print(f"Class {label}: {count} samples")

## Combining training and validation sets

In [ ]:
import numpy as np
import os

# Novo diretório onde os arquivos combinados serão salvos
cached_data_dir = '/content/drive/MyDrive/cubsat'
os.makedirs(cached_data_dir, exist_ok=True)

# Caminhos dos arquivos .npy combinados
all_images_path = os.path.join(cached_data_dir, 'all_images.npy')
all_labels_path = os.path.join(cached_data_dir, 'all_labels.npy')

# Verifica se os arquivos já existem
if os.path.exists(all_images_path) and os.path.exists(all_labels_path):
    print("Combined files already exist. Loading from disk...")
    all_images = np.load(all_images_path)
    all_labels = np.load(all_labels_path)
else:
    print("Combined files do not exist. Concatenating data...")
    # Concatena os conjuntos de treino e validação
    all_images = np.concatenate((train_images, val_images), axis=0)
    all_labels = np.concatenate((train_labels, val_labels), axis=0)

    # Salva os arquivos combinados
    np.save(all_images_path, all_images)
    np.save(all_labels_path, all_labels)
    print("Data concatenated and saved.")

# Exibe informações sobre as imagens
print("=== Image Characteristics ===")
print(f"Shape of combined dataset: {all_images.shape} (Samples, Height, Width, Channels)")
print(f"Image resolution: {all_images.shape[1:3]} pixels")
print(f"Total number of images: {all_images.shape[0]}")
print(f"Data type of images: {all_images.dtype}")
print(f"Minimum and maximum values in combined dataset: {all_images.min()} to {all_images.max()}")

# Exibe distribuição das classes
unique_labels, counts = np.unique(all_labels, return_counts=True)
print("\n=== Class Distribution in Combined Dataset ===")
for label, count in zip(unique_labels, counts):
    print(f"Class {label}: {count} samples")

print(f"\nData saved in: {cached_data_dir}")


In [ ]:
import numpy as np

# Caminho do arquivo
all_images_path = '/content/drive/MyDrive/cubsat/all_images.npy'

# Carregando o arquivo .npy
all_images = np.load(all_images_path)

# Verificando forma e tipo dos dados
print("Shape:", all_images.shape)
print("Dtype:", all_images.dtype)


PLOTANDO AMOSTRAS ALEATÓRIAS DE CADA CLASSE

In [ ]:
import numpy as np
import plotly.express as px
import random

# Caminhos dos arquivos
all_images_path = '/content/drive/MyDrive/cubsat/all_images.npy'
all_labels_path = '/content/drive/MyDrive/cubsat/all_labels.npy'

# Carregando os dados
all_images = np.load(all_images_path)
all_labels = np.load(all_labels_path)

# Identificando classes únicas
unique_classes = np.unique(all_labels)

# Plotando uma imagem aleatória de cada classe
for cls in unique_classes:
    # Índices das imagens da classe atual
    indices = np.where(all_labels == cls)[0]
    # Escolhendo um índice aleatório
    random_index = random.choice(indices)
    # Selecionando a imagem
    image = all_images[random_index]

    # Plotando com Plotly
    fig = px.imshow(image.astype(np.uint8))
    fig.update_layout(title=f"Class {cls}")
    fig.show()


## APLICANDO DATA AUGMENTATION PARA BALANCEAMENTO DE CLASSES

 Mostrar uma imagem aleatória com sua classe (após BALANCEAMENTO):

In [ ]:
import numpy as np
import os
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import shuffle

# Diretório dos arquivos originais
base_dir = '/content/drive/MyDrive/cubsat'
all_images_path = os.path.join(base_dir, 'all_images.npy')
all_labels_path = os.path.join(base_dir, 'all_labels.npy')

# Diretório para salvar dados balanceados
ros_dir = os.path.join(base_dir, "RandomOversampling")
os.makedirs(ros_dir, exist_ok=True)

# Caminhos dos arquivos balanceados
ros_images_path = os.path.join(ros_dir, "all_images_ros.npy")
ros_labels_path = os.path.join(ros_dir, "all_labels_ros.npy")

# Carregar dados originais
all_images = np.load(all_images_path)
all_labels = np.load(all_labels_path)

# Verifica se os arquivos já existem
if os.path.exists(ros_images_path) and os.path.exists(ros_labels_path):
    print("Random Oversampled files already exist. Loading from disk...")
    X_resampled = np.load(ros_images_path)
    y_resampled = np.load(ros_labels_path)
else:
    print("Applying Random Oversampling...")

    # Flatten temporariamente para aplicar o oversampling
    num_samples, height, width, channels = all_images.shape
    all_images_flat = all_images.reshape(num_samples, -1)

    # Aplicar RandomOverSampler
    ros = RandomOverSampler(sampling_strategy='auto', random_state=42)
    X_resampled_flat, y_resampled = ros.fit_resample(all_images_flat, all_labels)

    # Voltar ao formato original
    X_resampled = X_resampled_flat.reshape(-1, height, width, channels)

    # Embaralhar
    X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

    # Salvar
    np.save(ros_images_path, X_resampled)
    np.save(ros_labels_path, y_resampled)
    print("Random Oversampling applied and saved.")

# Mostrar distribuição das classes
unique_labels, counts = np.unique(y_resampled, return_counts=True)
print("\n=== Class Distribution after Random Oversampling ===")
for label, count in zip(unique_labels, counts):
    print(f"Class {label}: {count} samples")

print(f"\nBalanced data saved in: {ros_dir}")


In [ ]:
import numpy as np
import os
import plotly.express as px
import random

# Caminhos
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'
ros_images_path = os.path.join(ros_dir, "all_images_ros.npy")
ros_labels_path = os.path.join(ros_dir, "all_labels_ros.npy")

# Carregar dados
images = np.load(ros_images_path)
labels = np.load(ros_labels_path)

# Identificar classes únicas
unique_classes = np.unique(labels)

# Criar subplots em plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go

n_classes = len(unique_classes)
fig = make_subplots(rows=1, cols=n_classes, subplot_titles=[f"Class {cls}" for cls in unique_classes])

# Adicionar uma imagem aleatória de cada classe
for idx, cls in enumerate(unique_classes):
    indices = np.where(labels == cls)[0]
    selected_idx = random.choice(indices)
    img = images[selected_idx]

    fig.add_trace(
        go.Image(z=img.astype(np.uint8)),
        row=1, col=idx+1
    )

# Layout
fig.update_layout(height=300, width=200 * n_classes, title_text="Random Example from Each Class (Balanced Dataset)")
fig.show()


## Splitting combined set in proportion 80% for training and 20% for validation

In [ ]:
import numpy as np
import os
import random

# Caminhos
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'
ros_images_path = os.path.join(ros_dir, "all_images_ros.npy")
ros_labels_path = os.path.join(ros_dir, "all_labels_ros.npy")

# Carregar dados
images = np.load(ros_images_path)
labels = np.load(ros_labels_path)

# Identificar classes únicas
unique_classes = np.unique(labels)

# Listas para armazenar os dados divididos
train_images, train_labels = [], []
val_images, val_labels = [], []

# Garantir reprodução
random.seed(42)

# Dividir os dados mantendo o balanceamento
for cls in unique_classes:
    indices = np.where(labels == cls)[0]
    random.shuffle(indices.tolist())

    split_idx = int(0.8 * len(indices))
    train_idx = indices[:split_idx]
    val_idx = indices[split_idx:]

    train_images.append(images[train_idx])
    train_labels.append(labels[train_idx])
    val_images.append(images[val_idx])
    val_labels.append(labels[val_idx])

# Concatenar resultados
train_images = np.concatenate(train_images, axis=0)
train_labels = np.concatenate(train_labels, axis=0)
val_images = np.concatenate(val_images, axis=0)
val_labels = np.concatenate(val_labels, axis=0)

# Salvar os arquivos
np.save(os.path.join(ros_dir, "train_images_ros.npy"), train_images)
np.save(os.path.join(ros_dir, "train_labels_ros.npy"), train_labels)
np.save(os.path.join(ros_dir, "val_images_ros.npy"), val_images)
np.save(os.path.join(ros_dir, "val_labels_ros.npy"), val_labels)

print("Conjuntos salvos com sucesso!")


GERANDO IMAGENS ALEATORIAS DE CADA CLASSE POR CONJUNTO

In [ ]:
import numpy as np
import os
import random
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Caminho
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'
train_images = np.load(os.path.join(ros_dir, "train_images_ros.npy"))
train_labels = np.load(os.path.join(ros_dir, "train_labels_ros.npy"))

# Classes únicas
unique_classes = np.unique(train_labels)
n_classes = len(unique_classes)

# Criar subplots
fig = make_subplots(rows=1, cols=n_classes, subplot_titles=[f"Class {cls}" for cls in unique_classes])

# Adicionar imagem aleatória de cada classe
for idx, cls in enumerate(unique_classes):
    indices = np.where(train_labels == cls)[0]
    selected_idx = random.choice(indices)
    img = train_images[selected_idx]

    fig.add_trace(
        go.Image(z=img.astype(np.uint8)),
        row=1, col=idx+1
    )

fig.update_layout(height=300, width=200 * n_classes, title_text="Treinamento: Uma imagem aleatória por classe")
fig.show()


In [ ]:
import numpy as np
import os
import random
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Caminho
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'
val_images = np.load(os.path.join(ros_dir, "val_images_ros.npy"))
val_labels = np.load(os.path.join(ros_dir, "val_labels_ros.npy"))

# Classes únicas
unique_classes = np.unique(val_labels)
n_classes = len(unique_classes)

# Criar subplots
fig = make_subplots(rows=1, cols=n_classes, subplot_titles=[f"Class {cls}" for cls in unique_classes])

# Adicionar imagem aleatória de cada classe
for idx, cls in enumerate(unique_classes):
    indices = np.where(val_labels == cls)[0]
    selected_idx = random.choice(indices)
    img = val_images[selected_idx]

    fig.add_trace(
        go.Image(z=img.astype(np.uint8)),
        row=1, col=idx+1
    )

fig.update_layout(height=300, width=200 * n_classes, title_text="Validação: Uma imagem aleatória por classe")
fig.show()


In [ ]:
import numpy as np
import os
import collections

# Caminho
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'

# Carregar labels
train_labels = np.load(os.path.join(ros_dir, "train_labels_ros.npy"))
val_labels = np.load(os.path.join(ros_dir, "val_labels_ros.npy"))

# Contar classes
train_counts = collections.Counter(train_labels)
val_counts = collections.Counter(val_labels)

# Exibir resultados
print("Distribuição das classes no conjunto de TREINAMENTO:")
for cls, count in sorted(train_counts.items()):
    print(f"Classe {cls}: {count} imagens")

print("\nDistribuição das classes no conjunto de VALIDAÇÃO:")
for cls, count in sorted(val_counts.items()):
    print(f"Classe {cls}: {count} imagens")


In [ ]:
import numpy as np
import os

# Caminhos atualizados
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'
bin_edges_path = os.path.join(ros_dir, 'bin_edges_ros.npy')

# Carregar dados de treino e validação
train_images = np.load(os.path.join(ros_dir, 'train_images_ros.npy'))
train_labels = np.load(os.path.join(ros_dir, 'train_labels_ros.npy'))
val_images = np.load(os.path.join(ros_dir, 'val_images_ros.npy'))
val_labels = np.load(os.path.join(ros_dir, 'val_labels_ros.npy'))

# Verificar se o arquivo de bin_edges já existe
if os.path.exists(bin_edges_path):
    print("Arquivo de bin_edges já existe. Carregando do disco...")
    bin_edges = np.load(bin_edges_path, allow_pickle=True)
else:
    print("Arquivo de bin_edges não encontrado. Calculando quantis...")

    # Definir os níveis de quantis (decis -> 10 intervalos)
    quantile_levels = np.linspace(0, 1, num=11)
    bin_edges = []

    # Calcular os quantis para cada canal (R, G, B)
    for channel in range(3):
        channel_pixels = train_images[:, :, :, channel].flatten()
        edges = np.quantile(channel_pixels, quantile_levels)
        bin_edges.append(edges)

    # Salvar os bin_edges
    np.save(bin_edges_path, bin_edges)
    print("Quantis calculados e bin_edges salvos.")

# Mostrar os bin_edges
print("\n=== Bin Edges por Canal ===")
for channel, edges in enumerate(bin_edges):
    print(f"Canal {channel} (R={channel==0}, G={channel==1}, B={channel==2}): {edges}")


In [ ]:
import numpy as np
import os
import gc

# Caminho base
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'

# Caminhos dos arquivos
train_images_path = os.path.join(ros_dir, 'train_images_ros.npy')
train_labels_path = os.path.join(ros_dir, 'train_labels_ros.npy')
val_images_path = os.path.join(ros_dir, 'val_images_ros.npy')
val_labels_path = os.path.join(ros_dir, 'val_labels_ros.npy')
bin_edges_path = os.path.join(ros_dir, 'bin_edges_ros.npy')

# Carregar dados
print("Carregando dados de treino e validação...")
train_images = np.load(train_images_path)
train_labels = np.load(train_labels_path)
val_images = np.load(val_images_path)
val_labels = np.load(val_labels_path)

# Verificar existência de bin_edges
if os.path.exists(bin_edges_path):
    print("Arquivo bin_edges_ros.npy já existe. Carregando...")
    bin_edges = np.load(bin_edges_path, allow_pickle=True)
else:
    print("Calculando bin_edges (quantis)...")
    quantile_levels = np.linspace(0, 1, num=11)  # decílicos
    bin_edges = []
    for channel in range(3):  # R, G, B
        pixels = train_images[:, :, :, channel].flatten()
        edges = np.quantile(pixels, quantile_levels)
        bin_edges.append(edges)
    np.save(bin_edges_path, bin_edges)
    print("bin_edges salvos em disco.")

# Exibir bin_edges (opcional)
print("\n=== Bin Edges por Canal ===")
for ch, edges in enumerate(bin_edges):
    print(f"Canal {ch}: {edges}")

# Função para converter imagem em histograma normalizado
def image_to_histogram(image, bin_edges):
    features = []
    for channel in range(3):  # R, G, B
        pixels = image[:, :, channel].flatten()
        hist, _ = np.histogram(pixels, bins=bin_edges[channel])
        hist = hist / len(pixels)  # Normaliza para proporção
        features.extend(hist)
    return np.array(features)

# Converter imagens de treino em histogramas
print("\nConvertendo imagens de treino...")
train_histograms = np.array([image_to_histogram(img, bin_edges) for img in train_images])
print("Histograma de treino criado.")

# Liberar memória
del train_images
gc.collect()
print("train_images removido da memória.")

# Converter imagens de validação em histogramas
print("\nConvertendo imagens de validação...")
val_histograms = np.array([image_to_histogram(img, bin_edges) for img in val_images])
print("Histograma de validação criado.")

# Liberar memória
del val_images
gc.collect()
print("val_images removido da memória.")

# Salvar histogramas
print("\nSalvando histogramas em disco...")
np.save(os.path.join(ros_dir, "train_histograms_ros.npy"), train_histograms)
np.save(os.path.join(ros_dir, "val_histograms_ros.npy"), val_histograms)
print("Histogramas salvos com sucesso.")


---

### Train CubeSatNet DNN model

We will define and train a Dense Neural Network (DNN) model.

PARA GRAFICOS

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Caminho base
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'

# Carregar histogramas e rótulos
train_histograms = np.load(os.path.join(ros_dir, "train_histograms_ros.npy"))
val_histograms = np.load(os.path.join(ros_dir, "val_histograms_ros.npy"))
train_labels = np.load(os.path.join(ros_dir, "train_labels_ros.npy"))
val_labels = np.load(os.path.join(ros_dir, "val_labels_ros.npy"))

# Número de classes (assumindo 5, como no seu código original)
n_classes = len(np.unique(train_labels))

# One-hot encoding dos rótulos
train_labels_cat = to_categorical(train_labels, num_classes=n_classes)
val_labels_cat = to_categorical(val_labels, num_classes=n_classes)

# Definir o modelo
model = Sequential([
    Dense(128, activation='relu', input_shape=(train_histograms.shape[1],)),
    Dense(64, activation='relu'),
    Dense(n_classes, activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento com early stopping
history = model.fit(
    train_histograms, train_labels_cat,
    validation_data=(val_histograms, val_labels_cat),
    epochs=30,
    batch_size=32,
    callbacks=[EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)]
)

# Avaliar o modelo
loss, acc = model.evaluate(val_histograms, val_labels_cat, verbose=0)
print(f"\nAcurácia final na validação: {acc:.4f}")


In [ ]:
import plotly.graph_objects as go

# === Plot da Acurácia ===
fig_acc = go.Figure()
fig_acc.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines+markers', name='Treino'))
fig_acc.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines+markers', name='Validação'))
fig_acc.update_layout(
    title="Evolução da Acurácia",
    xaxis_title="Época",
    yaxis_title="Acurácia",
    yaxis=dict(range=[0, 1]),
    template="plotly_white"
)
fig_acc.show()

# === Plot da Loss ===
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines+markers', name='Treino'))
fig_loss.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines+markers', name='Validação'))
fig_loss.update_layout(
    title="Evolução da Loss",
    xaxis_title="Época",
    yaxis_title="Loss",
    yaxis=dict(range=[0, 1]),
    template="plotly_white"
)
fig_loss.show()


In [ ]:
import plotly.graph_objects as go

# === Plot da Acurácia ===
fig_acc = go.Figure()
fig_acc.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines+markers', name='Treino'))
fig_acc.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines+markers', name='Validação'))
fig_acc.update_layout(
    title="Evolução da Acurácia",
    xaxis_title="Época",
    yaxis_title="Acurácia",
    template="plotly_white"
)
fig_acc.show()

# === Plot da Loss ===
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines+markers', name='Treino'))
fig_loss.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines+markers', name='Validação'))
fig_loss.update_layout(
    title="Evolução da Loss",
    xaxis_title="Época",
    yaxis_title="Loss",
    template="plotly_white"
)
fig_loss.show()


Otimizando o codigo

In [ ]:
import numpy as np
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from kerastuner.tuners import RandomSearch
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tensorflow.keras.optimizers import Adam, SGD, RMSprop

# Caminho base
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'

# 1. Função para carregar os dados
def carregar_dados(ros_dir):
    train_X = np.load(os.path.join(ros_dir, "train_histograms_ros.npy"))
    val_X = np.load(os.path.join(ros_dir, "val_histograms_ros.npy"))
    train_y = np.load(os.path.join(ros_dir, "train_labels_ros.npy"))
    val_y = np.load(os.path.join(ros_dir, "val_labels_ros.npy"))
    return train_X, val_X, train_y, val_y

# 2. Função de criação do modelo com busca de hiperparâmetros
def construir_modelo_hp(hp):
    model = Sequential()

    model.add(Dense(
        units=hp.Int('units_hidden1', min_value=64, max_value=256, step=32),
        activation=hp.Choice('activation1', ['relu', 'tanh']),
        input_shape=(train_X.shape[1],)
    ))

    # Dropout opcional
    model.add(Dropout(hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(
        units=hp.Int('units_hidden2', min_value=32, max_value=128, step=32),
        activation=hp.Choice('activation2', ['relu', 'tanh'])
    ))

    model.add(Dropout(hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(n_classes, activation='softmax'))

    # Escolher otimizador
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    optimizer_name = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    if optimizer_name == 'adam':
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        optimizer = SGD(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# 3. Plot com Plotly
def plotar_historico_plotly(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=("Acurácia", "Loss"))

    fig.add_trace(go.Scatter(
        y=history.history['accuracy'], mode='lines+markers', name='Acurácia Treino'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        y=history.history['val_accuracy'], mode='lines+markers', name='Acurácia Validação'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        y=history.history['loss'], mode='lines+markers', name='Loss Treino'
    ), row=1, col=2)

    fig.add_trace(go.Scatter(
        y=history.history['val_loss'], mode='lines+markers', name='Loss Validação'
    ), row=1, col=2)

    fig.update_layout(
        title="Histórico de Treinamento",
        xaxis_title="Época",
        xaxis2_title="Época",
        yaxis_title="Acurácia",
        yaxis2_title="Loss",
        template="plotly_white",
        width=1000,
        height=400
    )
    fig.show()

# 4. Executar pipeline
train_X, val_X, train_y_raw, val_y_raw = carregar_dados(ros_dir)
n_classes = len(np.unique(train_y_raw))
train_y = to_categorical(train_y_raw, num_classes=n_classes)
val_y = to_categorical(val_y_raw, num_classes=n_classes)

# 5. Inicializar o tuner
tuner = RandomSearch(
    construir_modelo_hp,
    objective='val_accuracy',
    max_trials=20,
    executions_per_trial=1,
    directory='keras_tuner_dir',
    project_name='cubsat_tuning'
)

tuner.search_space_summary()

# 6. Treinar com early stopping
callbacks = [EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)]

tuner.search(train_X, train_y,
             validation_data=(val_X, val_y),
             epochs=30,
             batch_size=32,
             callbacks=callbacks,
             verbose=1)

# 7. Melhor modelo
best_model = tuner.get_best_models(num_models=1)[0]

# 8. Treinar melhor modelo final
history = best_model.fit(
    train_X, train_y,
    validation_data=(val_X, val_y),
    epochs=30,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

# 9. Avaliar e salvar
loss, acc = best_model.evaluate(val_X, val_y, verbose=0)
print(f"\nAcurácia final na validação: {acc:.4f}")

# Salvar melhor modelo
modelo_final_path = os.path.join(ros_dir, "modelo_melhor_otimizado.keras")
best_model.save(modelo_final_path)
print(f"Modelo salvo em: {modelo_final_path}")

# 10. Visualizar histórico
plotar_historico_plotly(history)


In [ ]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

# Caminho do modelo salvo
modelo_path = '/content/drive/MyDrive/cubsat/RandomOversampling/modelo_melhor_otimizado.keras'

# Carregar modelo salvo
modelo = load_model(modelo_path)

# Recarregar dados
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'
train_X = np.load(os.path.join(ros_dir, "train_histograms_ros.npy"))
val_X = np.load(os.path.join(ros_dir, "val_histograms_ros.npy"))
train_y_raw = np.load(os.path.join(ros_dir, "train_labels_ros.npy"))
val_y_raw = np.load(os.path.join(ros_dir, "val_labels_ros.npy"))

# One-hot encoding
n_classes = len(np.unique(train_y_raw))
train_y = to_categorical(train_y_raw, num_classes=n_classes)
val_y = to_categorical(val_y_raw, num_classes=n_classes)

# Treinamento por 50 épocas sem early stopping
history = modelo.fit(
    train_X, train_y,
    validation_data=(val_X, val_y),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Salvar o modelo reentreinado
novo_path = os.path.join(ros_dir, "modelo_melhor_otimizado_mais50epocas.keras")
modelo.save(novo_path)
print(f"Modelo reentreinado salvo em: {novo_path}")


In [ ]:
import plotly.graph_objects as go

# === Plot da Acurácia ===
fig_acc = go.Figure()
fig_acc.add_trace(go.Scatter(y=history.history['accuracy'], mode='lines+markers', name='Treino'))
fig_acc.add_trace(go.Scatter(y=history.history['val_accuracy'], mode='lines+markers', name='Validação'))
fig_acc.update_layout(
    title="Evolução da Acurácia",
    xaxis_title="Época",
    yaxis_title="Acurácia",
    yaxis=dict(range=[0, 1]),
    template="plotly_white"
)
fig_acc.show()

# === Plot da Loss ===
fig_loss = go.Figure()
fig_loss.add_trace(go.Scatter(y=history.history['loss'], mode='lines+markers', name='Treino'))
fig_loss.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines+markers', name='Validação'))
fig_loss.update_layout(
    title="Evolução da Loss",
    xaxis_title="Época",
    yaxis_title="Loss",
    yaxis=dict(range=[0, 1]),
    template="plotly_white"
)
fig_loss.show()


TESTANDO O MODELO COM O CONJUNTO DE TESTES

In [ ]:
import numpy as np
import os
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Caminhos
ros_dir = '/content/drive/MyDrive/cubsat/RandomOversampling'
test_images_path = '/content/drive/MyDrive/cubsat/test_images.npy'
test_labels_path = '/content/drive/MyDrive/cubsat/test_labels.npy'
bin_edges_path = os.path.join(ros_dir, 'bin_edges_ros.npy')
model_path = os.path.join(ros_dir, '/content/drive/MyDrive/cubsat/RandomOversampling/modelo_melhor_otimizado_mais50epocas.keras')

# Carregar dados e modelo
test_images = np.load(test_images_path)
test_labels = np.load(test_labels_path)
model = load_model(model_path)
bin_edges = np.load(bin_edges_path, allow_pickle=True)

# Função de extração de histogramas
def image_to_histogram(image, bin_edges):
    features = []
    for channel in range(3):
        pixels = image[:, :, channel].flatten()
        hist, _ = np.histogram(pixels, bins=bin_edges[channel])
        hist = hist / len(pixels)
        features.extend(hist)
    return np.array(features)

# Converter imagens de teste em histogramas
test_histograms = np.array([image_to_histogram(img, bin_edges) for img in test_images])

# Obter previsões
predictions = model.predict(test_histograms)
predicted_labels = np.argmax(predictions, axis=1)

# Matriz de confusão
conf_matrix = confusion_matrix(test_labels, predicted_labels)
labels = [f"Classe {i}" for i in range(conf_matrix.shape[0])]

# Plot com Plotly
fig = ff.create_annotated_heatmap(
    z=conf_matrix,
    x=labels,
    y=labels,
    colorscale='Blues',
    showscale=True
)
fig.update_layout(
    title="Matriz de Confusão - Conjunto de Teste",
    xaxis_title="Rótulo Previsto",
    yaxis_title="Rótulo Verdadeiro"
)
fig.show()


In [ ]:
from sklearn.metrics import classification_report

# Gerar o relatório
report = classification_report(test_labels, predicted_labels, digits=4)

# Imprimir
print("Classification Report:")
print(report)


In [ ]:
import time
import os
import psutil
import tracemalloc
from sklearn.metrics import accuracy_score, f1_score

# Início da medição de tempo, memória e CPU
start_time = time.time()
tracemalloc.start()
process = psutil.Process(os.getpid())
cpu_percent_start = psutil.cpu_percent(interval=None)

# Previsão
predictions = model.predict(test_histograms)
predicted_labels = np.argmax(predictions, axis=1)

# Métricas
accuracy = accuracy_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels, average='weighted')

# Término da medição
cpu_percent_end = psutil.cpu_percent(interval=None)
_, peak_memory = tracemalloc.get_traced_memory()
tracemalloc.stop()
end_time = time.time()

# Tamanho do código do modelo salvo
model_file_path = os.path.join(ros_dir, 'dnn_histogram_model.keras')
model_code_size = os.path.getsize(model_file_path) / 1024  # em KB

# Impressão formatada
print("Evaluation Time:        {:.4f} seconds".format(end_time - start_time))
print("Peak Memory Usage:      {:.2f} MB".format(peak_memory / (1024 * 1024)))
print("Average CPU Usage:      {:.2f}%".format((cpu_percent_start + cpu_percent_end) / 2))
print("Algorithm Code Size:    {:.2f} KB".format(model_code_size))
print("Accuracy:               {:.4f}".format(accuracy))
print("F1 Score:               {:.4f}".format(f1))
